In [ ]:
#
# test out cleaning up markdown files
#

In [2]:
#
# read the files
#
from pathlib import Path 
from loguru import logger

data_path = Path('data')
md_files = list(data_path.glob("*.md"))
logger.info("found %d markdown files" % len(md_files,))

2024-05-01 14:42:52.394 | INFO     | __main__:<module>:9 - found 869 markdown files


In [10]:
import markdown
from bs4 import BeautifulSoup

def markdown_to_text(markdown_str:str)->str:
    html = markdown.markdown(markdown_str)
    soup = BeautifulSoup(html, features='html.parser')
    lines = soup.get_text().split("\n")
    lines = [line.strip().lower() for line in lines if len(line) > 0]
    return "\n".join(lines)

In [11]:
input_file = md_files[0]
with open(input_file,mode="rt", encoding="utf-8") as f:
    data = f.read()
txt = markdown_to_text(data)
txt

"cdfm-20221130.md\nla proposition du gouvernement\nnotre proposition\non aurait besoin des garanties plus solides\ninquiétude pour la qualité\non va au délà de ce qu'on a fait par le passé\ncontre-positions?\nudem\n- automne de 2024\n- conditions qui doivent être réalisées\n- l'offre est intéressant\nlaval\n- faut être prudent\n- une occasion dans la crise\n- est-ce qu'il y a un risque que l'uq s'embarque dans la même direction\n- il faut leur donner quelque chose\ncontact priviligé\nfmoq -- tient beaucoup à cette solution\nil faut travailler avec eux\nles gains sont intéressants\nconclusion"

In [12]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=256,
    chunk_overlap=20
)
docs = text_splitter.create_documents([txt])

In [13]:
docs

[Document(page_content="cdfm-20221130.md\nla proposition du gouvernement\nnotre proposition\non aurait besoin des garanties plus solides\ninquiétude pour la qualité\non va au délà de ce qu'on a fait par le passé\ncontre-positions?\nudem\n- automne de 2024"),
 Document(page_content="- automne de 2024\n- conditions qui doivent être réalisées\n- l'offre est intéressant\nlaval\n- faut être prudent\n- une occasion dans la crise\n- est-ce qu'il y a un risque que l'uq s'embarque dans la même direction\n- il faut leur donner quelque chose"),
 Document(page_content='contact priviligé\nfmoq -- tient beaucoup à cette solution\nil faut travailler avec eux\nles gains sont intéressants\nconclusion')]

In [17]:
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
openai = OpenAI()

In [18]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
txts = [txt.page_content for txt in docs]
embeddings = embedding_model.embed_documents(txts)


In [26]:
#
# try langchain's markdown splitter
#
from typing import Any
from datetime import datetime
from langchain_text_splitters.markdown import MarkdownTextSplitter

splitter = MarkdownTextSplitter()
md_files = list(Path('data').glob("*.md"))
input_files = md_files[1:10]
texts = []
metadata : list[dict[str,Any]] = []
for input in input_files:
    with open(input) as f:
        txt = f.read()
    texts.append(txt)
    modified: datetime = datetime.fromtimestamp(input.stat().st_mtime)
    metadata.append({"filename":input.as_posix(),"created":f"{modified}"})
docs = splitter.create_documents(texts)
print(len(docs))
print(len(metadata))
for doc in docs:
    doc.metadata = {'filename':metadata[i]["filename"],
                    'created': metadata[i]['created'],
                    'id': f"{metadata[i]['filename']}{i:04d}"}
    i = i+ 1
print(docs[0].metadata)

11
9


IndexError: list index out of range